In [59]:
import pandas as pd

df = pd.read_csv('/home/satadisha/Desktop/GitProjects/ELTweetTracker/Collection of Tweets.csv', index_col='ID', header=0)
print df.columns.values.tolist()

count=0
for index, row in df.iterrows():
    if count<1:
        tweetText=row['Tweets']
        #capitalization module
        #if all words are capitalized:
        if tweetText.isupper():
            print "All caps module"
        else:
            print tweetText
            tweetWordList=tweetText.split()
            print tweetWordList
            #implement the consecutive caps without punctuation clause
        count+=1
    else:
        break

['# of RTs', 'HashTags', 'TweetIds', 'Date', 'Replier Ids', 'Retweeted Distinct Ids', 'Tweets', 'Urls in Tweet', 'Username', 'Unnamed: 10', 'Unnamed: 11']
U.S. Presidential Election series Trump, Clinton wins big, Rubio... 
['U.S.', 'Presidential', 'Election', 'series', 'Trump,', 'Clinton', 'wins', 'big,', 'Rubio...']
